# Loading Data

Spark SQL (and its Datasets) allow us to load datasets and build a schema automatically. Let's do this with our NAM dataset.

In [ ]:
df = spark.read.load('hdfs://orion12:9001/nam/2019/11/*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')


In [ ]:
# How many records do we have?
df.count()

In [ ]:
# Let's take a look at the data. We can see our schema here:
df.take(3)

# Filtering, SQL

In [ ]:
# Let's say we want to get very high temperature values:
hot = df.filter(df.temperature_surface > 316)
hot.take(3)

In [ ]:
hot.count()

In [ ]:
# Creating an SQL 'table'
df.createOrReplaceTempView("my_table")

# For queries that will return a single result, .take(1) is very useful:
spark.sql('SELECT AVG(temperature_surface) FROM my_table').take(1)

In [ ]:
spark.sql('SELECT MAX(temperature_surface) FROM my_table').take(1)

In [ ]:
spark.sql('SELECT AVG(relative_humidity_zerodegc_isotherm) FROM my_table WHERE temperature_surface > 318').take(1)

In [ ]:
humidities = spark.sql('SELECT relative_humidity_zerodegc_isotherm FROM my_table WHERE temperature_surface > 318')

# Let's say we're going to use the DataSet generated above frequently.
# We can cache it here to get better performance for subsequent usages.
humidities.cache()
humidities.count()

In [ ]:
# This is probably okay to do here since we only have ~100 values
# to collect to the client:
local_hum = humidities.collect()

# (just be careful not to do it with huge datasets)

In [ ]:
for i in local_hum:
    print(i)

# Sampling

In [ ]:
samp = df.sample(False, .1)
print(samp.count())

# This will write the sample to '/sampled_output' in HDFS, one for each
# worker. You can call samp.coalesce(1) to bring all the data to a single
# worker before writing, but performance will suffer.
samp.write.csv('hdfs://orion12:9001/sampled_output', sep='\t', header=True)